In [1]:
import random
import torch
import torchmetrics

import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [3]:
train_features = np.load("../data/processed/balance_16k/model_input/train_features.npy")
train_labels = np.load("../data/processed/balance_16k/model_input/train_labels.npy")

test_features = np.load("../data/processed/balance_16k/model_input/test_features.npy")
test_labels = np.load("../data/processed/balance_16k/model_input/test_labels.npy")


In [4]:
train_features = torch.from_numpy(train_features).float()
train_features = train_features.to(device)
train_labels = torch.from_numpy(train_labels)
train_labels = train_labels.to(device)

test_features = torch.from_numpy(test_features).float()
test_features = test_features.to(device)
test_labels = torch.from_numpy(test_labels)
test_labels = test_labels.to(device)

In [5]:
print(f"Train features: {train_features.shape}, Train label: {train_labels.shape} ")
print(f"Test features: {test_features.shape}, Test label: {test_labels.shape} ")
train_min, train_max = train_features.min(), train_features.max()
print((train_min, train_max))

# inputs = 2 * (inputs - x_min) / (x_max - x_min) -1

Train features: torch.Size([14672, 3, 50, 50]), Train label: torch.Size([14672]) 
Test features: torch.Size([1631, 3, 50, 50]), Test label: torch.Size([1631]) 
(tensor(0., device='cuda:0'), tensor(13373320., device='cuda:0'))


In [6]:
batch_size = 32
dataset = TensorDataset(train_features, train_labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [7]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=2, padding=1)
        # self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(
            in_channels=32, out_channels=64, kernel_size=2, padding=1
        )
        # self.bn2 = nn.BatchNorm2d(64) 

        self.conv3 = nn.Conv2d(
            in_channels=64, out_channels=128, kernel_size=2, padding=1
        )

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        # self.bn3 = nn.BatchNorm2d(128)

        # self.dropout = nn.Dropout(p=0.25)

        self.fc1 = nn.Linear(128 * 7 * 7, 256)
        self.bn_fc1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 128)
        self.bn_fc2 = nn.BatchNorm1d(128)
        self.fc3 = nn.Linear(128, 6)

    def forward(self, x):
        x = self.pool(F.leaky_relu(self.conv1(x)))  # Conv1 -> -> leaky_relu -> Pool
        x = self.pool(F.tanh((self.conv2(x))))  # Conv2 -> BN -> Tanh -> Pool
        x = self.pool(F.tanh((self.conv3(x))))  # Conv3 -> BN -> Tanh -> Pool
        x = x.view(-1, 128 * 7 * 7)
        x = F.tanh(self.bn_fc1(self.fc1(x)))  # FC1 -> BN -> Tanh
        x = F.tanh(self.bn_fc2(self.fc2(x)))  # FC2 -> BN -> Tanh
        x = self.fc3(x)
        return x

In [8]:
model = CNN()
model = model.to(device=device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Rprop(model.parameters(), lr=0.0008)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

initial_lr = 0.0008  
base_lr = 0.0008  
warmup_steps = 10 
epochs = 20

In [9]:
def train_model(
    model=model,
    dataloader=dataloader,
    device=device,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    epochs=epochs,
    warmup_steps=warmup_steps,
    initial_lr=initial_lr,
    base_lr=base_lr,
):
    model = model.to(device=device)

    for epoch in range(epochs):
        if epoch < warmup_steps:
            lr = initial_lr + (base_lr - initial_lr) * (epoch / warmup_steps)
            for param_group in optimizer.param_groups:
                param_group["lr"] = lr
        else:
            scheduler.step()
            lr = scheduler.get_last_lr()[0]

        batch_loss = 0.0
        model.train()
        for batch_inputs, batch_targets in dataloader:
            batch_inputs = batch_inputs.to(device)
            batch_targets = batch_targets.to(device)

            optimizer.zero_grad()
            outputs = model(batch_inputs)
            loss = criterion(outputs, batch_targets)
            loss.backward()
            optimizer.step()

            batch_loss += loss.item()

        avg_loss = batch_loss / len(dataloader)
        print(
            f"Epoch {epoch+1}/{epochs}, Cross Entropy Loss: {avg_loss:.4f}, Learning Rate: {lr:.6f}"
        )

In [10]:
train_model()

Epoch 1/20, Cross Entropy Loss: 1.2093, Learning Rate: 0.000800
Epoch 2/20, Cross Entropy Loss: 1.1379, Learning Rate: 0.000800
Epoch 3/20, Cross Entropy Loss: 1.1089, Learning Rate: 0.000800
Epoch 4/20, Cross Entropy Loss: 1.0801, Learning Rate: 0.000800
Epoch 5/20, Cross Entropy Loss: 1.0576, Learning Rate: 0.000800
Epoch 6/20, Cross Entropy Loss: 1.0380, Learning Rate: 0.000800
Epoch 7/20, Cross Entropy Loss: 1.0149, Learning Rate: 0.000800
Epoch 8/20, Cross Entropy Loss: 0.9917, Learning Rate: 0.000800
Epoch 9/20, Cross Entropy Loss: 0.9744, Learning Rate: 0.000800
Epoch 10/20, Cross Entropy Loss: 0.9564, Learning Rate: 0.000800
Epoch 11/20, Cross Entropy Loss: 0.9450, Learning Rate: 0.000800
Epoch 12/20, Cross Entropy Loss: 0.9288, Learning Rate: 0.000800
Epoch 13/20, Cross Entropy Loss: 0.9128, Learning Rate: 0.000800
Epoch 14/20, Cross Entropy Loss: 0.9023, Learning Rate: 0.000800
Epoch 15/20, Cross Entropy Loss: 0.8898, Learning Rate: 0.000800
Epoch 16/20, Cross Entropy Loss: 0

In [11]:
model.eval()

test_dataset = TensorDataset(test_features, test_labels)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=6).to(device)

with torch.no_grad():
    for batch_test_inputs, batch_test_targets in test_dataloader:
        outputs = model(batch_test_inputs)
        predictions = torch.argmax(outputs, dim=1)
        accuracy.update(predictions, batch_test_targets)

final_accuracy = accuracy.compute().item()
print(f"Accuracy on the test data: {final_accuracy * 100:.2f}%")

accuracy.reset()

Accuracy on the test data: 67.81%
